In [53]:
#These libraries must be installed to the instance/VM in order to run correctly (gdal a requirement for )
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas

#Importing GIS Software
from rasterio import windows
from rasterio.features import shapes
import geopandas as gpd
import rasterio 
import gdal 
import os

#Importing other libraries
from itertools import product
from tqdm.autonotebook import tqdm
import multiprocess
import threading
import concurrent
import numpy as np


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [40]:
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/2018-07 La Paz/Site 5/lap_2018-07_site05_120m_RGB_cc.tif" .

In [5]:
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/2019-05 Sian Ka'an Reserve/Site 1/skr_2019-05_site01_rgb_classifications/downsampled_m_skr_2019-05_site01_rgb.tif" .

In [66]:
'''
fix_gdalshp

Fixes shapefiles or geopandas databases generated from gdal_polygonize.py to output geometries that only contain the image of interest. Note that this script either takes an input geopandas database, or filename,
NOT BOTH

Inputs- 

shp: geopandas dataframe of the shapefile outputted from gdal_polygonize.py
filename: of input shapefile from gdal_polygonize.py

Outputs: 
shp: geopandas dataframe of fixed polygons

'''

def fix_gdalshp(shp=None,filename=None):
    if (type(shp) == None) and (filename != None):
        shp = geopandas.read_file(filename)
    for index, feature in tqdm(shp.iterrows()):
        if feature["DN"] == 0:
            shp.drop(index, inplace=True)
    if (filename != None):
        shp.to_file(filename)
    return shp

'''
load_image

Inputs-

file(str): full path of image/orthmosaic

Outputs-

img: generator of original image/orthomosaic
meta(dict): contains meta information from image, including location, size, etc. 

'''

#for loading orthomosaic into memory 
def load_image(file):
    img = rasterio.open(file)
    meta = img.meta.copy()
    return img, meta



'''

get_tiles

Inputs-

ds: generator of image/orthomosaic, should be img returned from load_image()
width,height(int) = width and height of output tiles

Outputs-

out_window(list): 
out_transform(list): contains meta information from image, including location, size, etc. 

original get_tiles implementation from 

https://gis.stackexchange.com/questions/285499/how-to-split-multiband-image-into-image-tiles-using-rasterio

'''

#gets the windows and transforms of all tiles within an orthomosaic
def get_tiles(ds, width=256, height=256):
    out_window = []
    out_transform = []
    ncols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, ncols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=ncols, height=nrows)
    for col_off, row_off in  offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        out_window.append(window)
        out_transform.append(windows.transform(window, ds.transform))
    return out_window, out_transform

'''

retile

inputs:
img: generator from input image or orthmosaic
meta(dict): contains meta information from image, including location, size, etc. 

outputs(list): list of numpy arrays of input tiles

'''
def retile(img, meta, out_path = 'images/', files=False, width=256, height=256):

    #getting tiles and setting filenames for the outputy files
    output_filename = 'tile_{}-{}.tif'
    window, transform = get_tiles(img, width, height)

    #locking read and write since they are not thread safe 
    read_lock = threading.Lock()
    write_lock = threading.Lock()

    #creating process to be threaded
    def process(window,transform):

        #thread locking reading (not thread safe)
        with read_lock:
            tile = img.read(window=window)
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
        #checking for tiles that are only alpha band

        #thread locking writing (not thread safe)
        with write_lock:
            #if you want to write files 
            if files:
                outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(img.read(window=window))
        return tile

    results = []

    #iterating through the different windows and tranforms generated with get_tiles
    for cur_w, cur_t in tqdm(zip(window, transform)):
        #running the process above with the maximum amount of workers available and returning the future (result returned) 
        with concurrent.futures.ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            future = executor.submit(process, cur_w, cur_t)
        results.append(future.result())
    return results

'''

polygonize

works similar to gdal_polygonize, but much faster :)

Inputs- 

img: generator from input image or orthmosaic
out_file: output filename of shapefile to write

Outputs: geopandas_df: geopandas dataframe containing the geometries from the input raster band

'''

def polygonize(img, out_file=None, band=4):
    raster = img.read(band)
    geometries = list((
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(raster, mask=None, transform=img.transform))))
    
    geopandas_df  = gpd.GeoDataFrame.from_features(geometries)

    if out_file != None: 
            geopandas_df.to_file(out_file)

    return geopandas_df

'''

fix_shp

works similar to gdal_polygonize, but much faster :)

Inputs- 

img: generator from input image or orthmosaic
filename: filename of input shapefile generated from gisutils 

Outputs-
shp: geopandas dataframe of fixed polygons


'''


def fix_shp(shp=None,filename=None):
    if (type(shp) == None) and (filename != None):
        shp = geopandas.read_file(filename)
    for index, feature in tqdm(shp.iterrows()):
        if feature["raster_val"] == 0:
            shp.drop(index, inplace=True)
    
    if (filename != None):
        shp.to_file(filename)
    return shp


#def get_area(geopandas_df):

In [ ]:
# 16 seconds - intial implementation
# 13 seconds - gdal_retile.py
# 12.5 seconds - retile implementation
# 8.25 seconds - no file saving

In [ ]:
#example usage of retile

%%time
file = "lap_2018-07_site05_120m_RGB_cc.tif" 
img, meta = load_image(file)
results = generate_tiles(img,meta,files=False)

CPU times: user 5.92 s, sys: 1.2 s, total: 7.12 s
Wall time: 8.24 s


In [42]:
!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/2018-05 Puerto San Carlos/Site 9/psc_2018-05_site09_120m_RGB_classifications/m_psc_2018-05_site09_120m_RGB.tif" .

In [ ]:
#example usage of polygonize and fixshp

%%time
file = "/content/downsampled_m_psc_2018-05_site09_120m_RGB.tif"
img, meta = load_image(file)
df = polygonize(img,  out_file="/content/test.shp", band=4)
#df = fix_shp(shp=df)


CPU times: user 26.3 ms, sys: 3.72 ms, total: 30 ms
Wall time: 32.5 ms
